In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR, MSOL
from holodeck import utils, detstats, plot


import hasasia.sensitivity as hsen
import hasasia.sim as hsim

# Read in strains from hdf

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
ssfile.close()

shape = hc_ss.shape
nsamps, nfreqs, nreals, nlouds = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L =',nsamps, nfreqs, nreals, nlouds)

# Find frequency bin for 1/yr

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

fidx = (np.abs(fobs - 1/YR)).argmin()
print('fobs[%d] = %f /yr' % (fidx, fobs[fidx]*YR))

# Value to Match To
The 12.5 yr paper quotes a media common process amplitude at f=1/yr of $A_\mathrm{CP} \cong 1.92\times10^{-15}$ with fixed DE438 ephemeris, or $A_\mathrm{CP} \cong 1.53\times10^{-15}$ with BAYESEPHEM corrections to DE438

I know $h_c \propto A_\mathrm{yr^{-1}} (\frac{f}{1 \mathrm{yr^{-1}}})^{-2/3}$ but not sure about scale factor. If it's same as the amplitude we calculated earlier, $A = \frac{10^{1/2}}{4} h_s = \frac{10^{1/2}}{4} h_c * \sqrt{\Delta f_{obs} / f_{obs}} $ 

The current astro interp paper Eq. (17) uses turnover model eq.
$$ h_c(f) = A_\mathrm{yr} \frac{(f\cdot \mathrm{yr})^{-2/3}}{(1+(f_b/f)^\kappa)^{1/2}} $$

15yr GWB paper uses strain amplitude $2.4 \times 10^{-15}$

So, I'll assume the 12.5 yr number is strain amplitude, and multiply that by $\sqrt{\frac{f}{df}}$ to get h_c

In [ ]:
A_ref = 1.53*10**-15
hc_ref = A_ref * np.sqrt(fobs[fidx]/dfobs[fidx])
print(hc_ref)

In [ ]:
# median strain at 1/yr amplitude, for each sample
hc_1yr = np.sqrt(hc_bg[:,fidx,:]**2 + np.sum(hc_ss[:,fidx,:,:]**2, axis=-1)) # (N,R)
hc_1yr = np.median(hc_1yr, axis=1) 
print(hc_1yr.shape)
plt.plot(np.arange(nsamps), hc_1yr)
plt.yscale('log')

In [ ]:
nsort = np.argsort(np.abs(hc_1yr-hc_ref))

plt.scatter(np.arange(nsamps), hc_1yr[nsort], s=3)
plt.yscale('log')

In [ ]:
def amp_to_hc(amp_ref, fobs, dfobs):
    hc = amp_ref*np.sqrt(fobs/dfobs)
    return hc

def rank_samples(hc_ss, hc_bg, fobs, dfobs, amp_ref):
    """ Sort samples by those with f=1/yr char strains closest to some reference value.
    
    Parameters
    ----------
    hc_ss : (N,F,R,L) NDarray
        Characteristic strain of the loudest single sources.
    hc_bg : (N,F,R) NDarray
        Characteristic strain of the background.
    fobs : (F,)
        Observed GW frequency
    dfobs : (F,)
        Observed GW frequency bin widths.
    amp_ref : scalar
        Reference strain amplitude at f=1/yr

    Returns
    -------
    nsort : (N,) 1Darray
        Indices of the param space samples sorted by proximity to the reference 1yr amplitude.
    """
    # find bin with 1/yr
    fidx = (np.abs(fobs - 1/YR)).argmin()

    # find reference (e.g. 12.5 yr) char strain
    hc_ref = amp_to_hc(amp_ref, fobs[fidx], dfobs[fidx])

    # select 1/yr median strains of samples
    hc_1yr = np.sqrt(hc_bg[:,fidx,:]**2 + np.sum(hc_ss[:,fidx,:,:]**2, axis=-1)) # (N,R)
    hc_1yr = np.median(hc_1yr, axis=1) 

    # sort by closest
    nsort = np.argsort(np.abs(hc_1yr-hc_ref))

    return nsort